# 🤖 Chatbot using Machine Learning (Jupyter Notebook)

In [1]:
!pip install tensorflow nltk scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...


True

In [4]:
lemmatizer = WordNetLemmatizer()

with open('intents.json') as file:
    data = json.load(file)

texts = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        texts.append(pattern)
        tags.append(intent['tag'])

lbl_enc = LabelEncoder()
labels = lbl_enc.fit_transform(tags)

with open('labels.pkl', 'wb') as f:
    pickle.dump(lbl_enc, f)

tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

with open('words.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, padding='post', maxlen=10)
y = np.array(labels)

In [5]:
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16, input_length=10))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(len(set(y)), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=200)
model.save("chatbot_model.h5")

c:\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.4444 - loss: 1.0917
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.4444 - loss: 1.0957
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - accuracy: 0.2222 - loss: 1.1003
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 0.5556 - loss: 1.0916
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.5556 - loss: 1.0905
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.4444 - loss: 1.0849
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.4444 - loss: 1.0956
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.4444 - loss: 1.0926
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step - accuracy: 0.4444 - loss: 1.0773
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.4444 - loss: 1.0900
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.4444 - loss: 1.0807
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("chatbot_model.h5")

with open("intents.json") as file:
    intents = json.load(file)

tokenizer = pickle.load(open("words.pkl", "rb"))
lbl_enc = pickle.load(open("labels.pkl", "rb"))

def chat():
    print("Chatbot is ready! Type 'quit' to exit.")
    while True:
        msg = input("You: ")
        if msg.lower() == 'quit':
            break
        seq = tokenizer.texts_to_sequences([msg])
        padded = pad_sequences(seq, truncating='post', maxlen=10)
        pred = model.predict(padded)[0]
        tag = lbl_enc.inverse_transform([np.argmax(pred)])[0]
        for intent in intents['intents']:
            if intent['tag'] == tag:
                print("Bot:", random.choice(intent['responses']))

chat()

Chatbot is ready! Type 'quit' to exit.


You:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
Bot: Greetings!
